# Recursion

## Outline

- Motivation

- What is recursion

- Examples of recursion

In [ ]:
-- | Applied to a predicate and a list, 'any' determines if any element
-- of the list satisfies the predicate.  For the result to be
-- 'False', the list must be finite; 'True', however, results from a 'True'
-- value for the predicate applied to an element at a finite index of a finite or infinite list.
any                     :: (a -> Bool) -> [a] -> Bool
any _ []        = False
any p (x:xs)    = p x || any p xs

In [ ]:
filter :: (a -> Bool) -> [a] -> [a]
filter _pred []    = []
filter pred (x:xs)
  | pred x         = x : filter pred xs
  | otherwise      = filter pred xs

In [ ]:
take n _      | n <= 0 =  []
take _ []              =  []
take n (x:xs)          =  x : take (n-1) xs

In [ ]:
takeWhile               :: (a -> Bool) -> [a] -> [a]
takeWhile _ []          =  []
takeWhile p (x:xs)
            | p x       =  x : takeWhile p xs
            | otherwise =  []

In [ ]:
zip :: [a] -> [b] -> [(a,b)]
zip []     _bs    = []
zip _as    []     = []
zip (a:as) (b:bs) = (a,b) : zip as bs

In [ ]:
zipWith :: (a->b->c) -> [a]->[b]->[c]
zipWith _f []     _bs    = []
zipWith _f _as    []     = []
zipWith f  (a:as) (b:bs) = f a b : zipWith f as bs

In [ ]:
reverse' :: [a] -> [a] 
reverse' l =  rev l []
  where
    rev []     a = a
    rev (x:xs) a = rev xs (x:a)

Identify the pattern that keeps repeating (base case and list with contents) using reverse' and present foldl as the abstraction of it.

In [ ]:
-- if the list is empty, the result is the initial value; else
-- we recurse immediately, making the new initial value the result
-- of combining the old initial value with the first element.
foldl' :: (a -> b -> a) -> a -> [b] -> a
foldl' f acc [] = acc
foldl' f acc (x:xs) = foldl' f (f acc x) xs

Replace the abstracted pattern

In [ ]:
reverse' :: [a] -> [a] 
reverse' l =  rev l []
  where
    rev []     a = a
    rev (x:xs) a = rev xs (x:a)

reverse'' :: [a] -> [a]  
reverse'' = foldl (\acc x -> x : acc) [] 

In [ ]:
map :: (a -> b) -> [a] -> [b]
map _ []     = []
map f (x:xs) = f x : map f xs

Identify abstract pattern foldr

In [ ]:
-- if the list is empty, the result is the initial value acc; else
-- apply f to the first element and the result of folding the rest
foldr' :: (a -> b -> b) -> b -> [a] -> b
foldr' f acc [] = acc
foldr' f acc (x:xs) = f x (foldr' f acc xs)

Replace the abstracted pattern

In [ ]:
map' :: (a -> b) -> [a] -> [b]
map' _ []     = []
map' f (x:xs) = f x : map f xs
 
map'' :: (a -> b) -> [a] -> [b]  
map'' f xs = foldr (\x acc -> f x : acc) [] xs  

## Motivation

Haskell does not have built-in looping constructs as `for` or `while` loops that can be found in many other programming languages. However, we can't live without cycles in code. So, we use `recursion` instead.

## What is recursion

Recursion in programming occurs when a function calls itself. That means the function name is used somewhere in the function body. It is a way to construct loops. Recursion in Haskell is made simpler when using pattern matching. Defining functions with pattern matching offers an elegant way to define when the recursive loop stops.

Here is a basic example of recursion with and without pattern matching. We define a function called `myProduct` that takes a list of type `[Int]` and returns the product of its elements. For an empty list, the function should return 0. For the second variant, we use the build in function `head` and `tail` that return the first element of the list and the list without the first element:
```haskell
head :: [a] -> a 
tail :: [a] -> [a] 
```

In [ ]:
myProduct :: [Int] -> Int
myProduct [] = 0
myProduct [x] = x
myProduct (x:xs) = x * myProduct xs

myProduct' :: [Int] -> Int
myProduct' xs = if xs == [] then 0
               else multiply xs 
  where multiply xs = if xs == [] then 1
                      else head xs * multiply (tail xs)

We see that the solution in the function `myProduct` that uses pattern matching is much more elegant as the one in the function `myProduct'` that does not use it. The things you have to be careful when writing recursive functions is that they come to an end and when using pattern matching that the patterns are not non-exhaustive. 

For instance, the pattern match in the `myProduct` function above would be non-exhaustive if you would skip the line `myProduct [] = 0`. The compiler would still accept your code, but the function would not work for all user inputs. For `[]` it would return an error. 

## Recursion examples

Let's write our own version of some simple Haskell functions with the use of recursion and pattern matching and test them against the Haskell built-in functions: 

- the function `length` returns the length of a list

- the function `take n` returns the first n elements of a list

- the function `sum` sums the elements of a list

In [ ]:
myLength [] = 0
myLength (x:xs) = 1 + myLength xs

myLength [1..3] == length [1..3]

myTake _ [] = []
myTake 0 _ = []
myTake n (x:xs) = x : myTake (n-1) xs

myTake 3 [1..5] == take 3 [1..5]
myTake 7 [1..5] == take 7 [1..5]

mySum [x] = x
mySum (x:xs) = x + (mySum xs)

mySum [1..3] == sum [1..3]

A more complicated example of recursion is if we try to implement the function `sort` that sorts the elements of a list. Here we use beside the functions `head` and `tail` also the `minimum` function that returns the smallest number of a list that contains numbers.

In [ ]:
mySort :: [Int] -> [Int]
mySort [] = []
mySort xs = minimum xs : mySort (removeMin xs)
  where removeMin ys = let minYs = minimum ys
                       in removeMin' minYs ys
        removeMin' minYs [] = []
        removeMin' minYs ys = if head ys == minYs 
                              then removeMin' minYs (tail ys)
                              else head ys : removeMin' minYs (tail ys)

print $ mySort [2,6,1,7,3,0,3]

The reason we did not do a comparison with the actual `sort` function is that our function does not preserve repeated elements and the `sort` function does.

For our last example of recursion, we implement the function `fib` that takes in the number `n` of type `Int` and return the first n elements of the Fibonacci list (https://en.wikipedia.org/wiki/Fibonacci_number). Here we use the functions `last` and `init` that return the last element of the list and all the elements of the list except the last.
```haskell
last :: [a] -> a 
init :: [a] -> [a] 
```

In [ ]:
fib :: Int -> [Int]
fib n = addElement [1,1]
  where addElement xs = 
          if length xs == n then xs 
          else addElement (xs ++ [last xs + last (init xs)])

print $ fib 10